## Taxonomies distribution across data entries

Entries are duplicated because split across different taxonomies present in the collection. This is a preparatory analysis if we want to keep working on time curves. Time curves will traverse one ta

In [6]:
import json
import re

content_list_filepath = '/Users/francescamorini/projects/dmacp/dmacp_data/article_list.json'

projects_data_filepath = '/Users/francescamorini/projects/dmacp/dmacp_data/article_list.json'

merged_content_data = {
    'contribution': {
        'filepath': '/Users/francescamorini/projects/dmacp/dmacp_data/contributions.json',
        'data': {}
    },
    'field_note': {
        'filepath': '/Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json',
        'data': {}
    },
    'project': {
        'filepath': '/Users/francescamorini/projects/dmacp/dmacp_data/projects.json',
        'data': {}
    }
}

# Helper functions

def get_text_from_html(raw_html):
    
    # from https://stackoverflow.com/a/12982689
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    
    cleantext = cleantext.replace('&nbsp;', ' ')
    cleantext = cleantext.replace('\n\n', ' ')
    cleantext = cleantext.replace('\r\n', ' ')
    cleantext = cleantext.replace('\n', ' ')    
    cleantext = cleantext.strip()
    
    return cleantext

def get_embed_content_entry(entry_data):
    if entry_data['title'] or entry_data['caption']:
        return {
            'type': entry_data['acf_fc_layout'],
            'title': entry_data['title'],
            'text': entry_data['caption'],
        }
    
    return {}

def get_text_content_entry(entry_data):
    text = get_text_from_html(entry_data['text'])
    
    if text:
        return {
            'type': entry_data['acf_fc_layout'],
            'text': text,
        }

    return {}

def get_image_content_entry(entry_data):    
    text_elements = []
    for image_data in entry_data['image']:
        if image_data['caption']:
            text_elements.append(image_data['caption'].strip())
        if image_data['description']:
            text_elements.append(image_data['description'].strip())
    
    text = ' '.join(text_elements)
    
    if text:
        return {
            'type': entry_data['acf_fc_layout'],
            'text': get_text_from_html(text)
        }
    
    return {}
    

def get_essentialize_contribution_data(raw_data):
    return get_essentialize_project_data(raw_data)

def get_essentialize_field_note_data(raw_data):
    data = {}

    data['id'] = raw_data['id']

    data['link'] = 'https://notes.anthropocene-curriculum.org/id/%s' % raw_data['id']
    
    data['text'] = get_text_from_html(raw_data['acf']['text'])

    
    # Meta datetime to have some kind of time for each case
    data['meta_datetime'] = raw_data['date_gmt']

    # Getting also keywords, folksonomies and methods to later build correlation with time
    data['keywords'] = raw_data['keyword']
    data['folksonomy'] = raw_data['folksonomy']
    data['methods'] = raw_data['method']

    return data

def get_essentialize_project_data(raw_data):
    data = {}
    data['id'] = raw_data['id']
    # title
    data['title'] = raw_data['title']['rendered']
    
    # subtitle
    data['subtitle'] = raw_data['acf']['subtitle']
    
    # abstract
    data['abstract'] = get_text_from_html(raw_data['acf']['abstract'])
    
    # link
    data['link'] = raw_data['link'].replace('content.', 'www.')

    # Meta datetime to have some kind of time for each case
    data['meta_datetime'] = raw_data['date_gmt']

    #Taxonomies
    
    data['keywords'] = raw_data['keyword']
    data['folksonomy'] = raw_data['folksonomy']
    data['methods'] = raw_data['method']

        
    # content
    data['content'] = []
    if raw_data['acf']['_content']['content']:
        for entry in raw_data['acf']['_content']['content']:
            
            ignored_types = ['iframe', 'featured', 'featured-children', 'reading-list', 'pdf']
        
            # Get content based on layout type
            content = {}
            if entry['acf_fc_layout'] == 'embed':
                content = get_embed_content_entry(entry)
            elif entry['acf_fc_layout'] == 'text':
                content = get_text_content_entry(entry)
            elif entry['acf_fc_layout'] == 'image':
                content = get_image_content_entry(entry)
                
            elif entry['acf_fc_layout'] in ignored_types:
                # doesn't hold any content. skipped for now
                continue

            else:
                print('Warning: Unhandled content type %s.' % entry['acf_fc_layout'])
            
            # Add content if not empty
            if content:
                data['content'].append(content)

    return data

# Merge content files

with open(content_list_filepath) as content_list_file:
    data = json.load(content_list_file)
    
    for entry in data:
        filepath = '/Users/francescamorini/projects/dmacp/dmacp_data/%s/%s.json' % (entry['type'], entry['slug'])
        
        with open(filepath) as json_file:
            json_data = json.load(json_file)
        
            essentialized_data = {}
            if entry['type'] == 'contribution':
                essentialized_data = get_essentialize_contribution_data(json_data[0])
            elif entry['type'] == 'field_note':
                essentialized_data = get_essentialize_field_note_data(json_data[0])
            elif entry['type'] == 'project':
                essentialized_data = get_essentialize_project_data(json_data[0])
        
            merged_content_data[entry['type']]['data'][entry['slug']] = essentialized_data

for category in merged_content_data:
    filepath = merged_content_data[category]['filepath']
    data = merged_content_data[category]['data']
    
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile, sort_keys=True, indent=4)

num_contributions = len(merged_content_data['contribution']['data'])
num_field_notes = len(merged_content_data['field_note']['data'])
num_projects = len(merged_content_data['project']['data'])
print('Merged %d contributions, %d field_notes, and %d projects.' % (num_contributions, num_field_notes, num_projects))

Merged 264 contributions, 288 field_notes, and 103 projects.


In [7]:
import json
import spacy
import pandas as pd

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# compressed content files
contributions_file = '/Users/francescamorini/projects/dmacp/dmacp_data/contributions.json'
field_notes_file = '/Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json'
projects_file = '/Users/francescamorini/projects/dmacp/dmacp_data/projects.json'
taxonomy_file = '/Users/francescamorini/projects/dmacp/dmacp_data/taxonomy_list.json'

# Some helper functions

def get_merged_contribution_text(contribution_data):
    texts = [contribution_data['abstract']]
    for entry in contribution_data['content']:
        texts.append(entry['text'])
    return ' '.join(texts)

def get_merged_field_note_text(field_note_data):
    return field_note_data['text']

def get_merged_project_text(project_data):
    texts = [project_data['abstract']]
    for entry in project_data['content']:
        texts.append(entry['text'])
    return ' '.join(texts)

In [8]:
import os.path
taxonomyLabels = []
with open(taxonomy_file) as taxonomy:
    taxonomy_list = json.load(taxonomy)
    for el in taxonomy_list:
        taxonomyLabels.append(el)

print(taxonomyLabels)

['methods', 'keywords', 'folksonomy']


In [9]:
#Helper function to check taxonomy content for each entry
def labelMethod(entryValue, currentTaxonomy):
    with open(taxonomy_file) as taxonomy:
        taxonomy_list = json.load(taxonomy)
        for taxonomy in taxonomy_list[currentTaxonomy]:

            if taxonomy['term_id'] == entryValue:
                return taxonomy['name']

In [ ]:
def runTextAnalysis(currentLabel, single_entry):
     if currentLabel in single_entry:
        for element in single_entry['content']:
            if element['text']:
                doc = nlp(element['text'])
                for entity in doc.ents:
                    if entity.label_ in temporalLabels:
                        return entity.text

In [16]:
allFiles = [contributions_file, field_notes_file, projects_file]
typeLabel = ['contribution', 'fieldnote', 'project']
temporalLabels = ['DATE', 'TIME']

index = 0

timedMethods = []
timedKeywords = []
timedFolksonomies = []
datasets = [timedMethods, timedKeywords, timedFolksonomies]

#Initial loops through types of taxonomy types
for taxonomy in taxonomyLabels:

    print('started', taxonomy)
    fileIndex = 0
    #Opening files
    for file in allFiles:
        with open(file) as f:

            entries = json.load(f)
            print('open', file)

              #Loop through entries
            for entry in entries:

                # print(entries[entry])
                single_entry = entries[entry]
                entry_id = single_entry['id']
                metatime = single_entry['meta_datetime']
                if 'title' in single_entry:
                    title = single_entry['title']
                    
                else:
                    title = typeLabel[fileIndex] + ' with no title'
                    
                for tax in single_entry[taxonomy]:
                    temporalEntities = []
                    taxonomy_id = tax
                    taxonomy_name = labelMethod(tax, taxonomy)
                    
                    abstractEntities = runTextAnalysis('abstract', single_entry)
                    temporalEntities.append(abstractEntities)
                    contentEntities = runTextAnalysis('content', single_entry)
                    temporalEntities.append(contentEntities)
#                     if 'abstract' in single_entry:
#                         if single_entry['abstract']:
#                             doc = nlp(single_entry['abstract'])
#                             for entity in doc.ents:
#                                 if entity.label_ in temporalLabels:
#                                     temporalEntities.append(entity.text)
                                    
#                     if 'content' in single_entry:
#                         for element in single_entry['content']:
#                             if element['text']:
#                                 doc = nlp(element['text'])
#                                 for entity in doc.ents:
#                                     if entity.label_ in temporalLabels:
#                                         temporalEntities.append(entity.text)
                    
                
                    datasets[index].append(
                        [
                            title, 
                            entry_id, 
                            metatime, 
                            taxonomy_name, 
                            typeLabel[fileIndex], 
                            taxonomy,
                            temporalEntities
                        ])
            fileIndex = fileIndex + 1
    index = index + 1
print('done', taxonomy)

started methods
open /Users/francescamorini/projects/dmacp/dmacp_data/contributions.json
open /Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json
open /Users/francescamorini/projects/dmacp/dmacp_data/projects.json
started keywords
open /Users/francescamorini/projects/dmacp/dmacp_data/contributions.json
open /Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json
open /Users/francescamorini/projects/dmacp/dmacp_data/projects.json
started folksonomy
open /Users/francescamorini/projects/dmacp/dmacp_data/contributions.json
open /Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json
open /Users/francescamorini/projects/dmacp/dmacp_data/projects.json
done folksonomy


In [22]:
methods = pd.DataFrame(datasets[0], columns=['title','id', 'time', 'taxonomy_name', 'type_of_entry', 'type_of_taxonomy', 'temporalEntities'])
keywords = pd.DataFrame(datasets[1], columns=['title','id', 'time', 'taxonomy_name', 'type_of_entry', 'type_of_taxonomy', 'temporalEntities'])
folksonomies = pd.DataFrame(datasets[2], columns=['title','id', 'time', 'taxonomy_name', 'type_of_entry', 'type_of_taxonomy', 'temporalEntities'])

In [18]:
keywords['id'].nunique()

512

In [19]:
methods.sort_values('title')

,title,id,time,taxonomy_name,type_of_entry,type_of_taxonomy,temporalEntities
75,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,Conversation,contribution,methods,[]
3,30 Days On/30 Days Off with Paul Perkinson,20478,2020-02-24T14:23:37,Conversation,contribution,methods,[his twenties]
4,A Brief History of Geoengineering,22468,2014-11-23T15:13:02,Film,contribution,methods,[]
5,A Caribbean Taste of Technology: Creolization ...,28250,2017-04-15T14:21:22,Case Study,contribution,methods,[]
6,A Caribbean Taste of Technology: Creolization ...,28250,2017-04-15T14:21:22,Reflection,contribution,methods,[]
...,...,...,...,...,...,...,...
450,fieldnote with no title,11986,2019-08-29T19:20:00,Teaching,fieldnote,methods,[]
257,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,Conversation,contribution,methods,"[2017, October 2019, Today, today, Halloween, ..."
258,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,Engagement,contribution,methods,"[2017, October 2019, Today, today, Halloween, ..."
259,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,Field Work,contribution,methods,"[2017, October 2019, Today, today, Halloween, ..."


In [20]:
keywords.sort_values('title')

,title,id,time,taxonomy_name,type_of_entry,type_of_taxonomy,temporalEntities
180,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,Water,contribution,keywords,[]
179,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,Pollution,contribution,keywords,[]
178,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,History,contribution,keywords,[]
177,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,Biodiversity,contribution,keywords,[]
16,30 Days On/30 Days Off with Paul Perkinson,20478,2020-02-24T14:23:37,Pollution,contribution,keywords,[his twenties]
...,...,...,...,...,...,...,...
558,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,History,contribution,keywords,"[2017, October 2019, Today, today, Halloween, ..."
559,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,Inequality,contribution,keywords,"[2017, October 2019, Today, today, Halloween, ..."
560,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,Race,contribution,keywords,"[2017, October 2019, Today, today, Halloween, ..."
562,“Planting a Seed is a Revolutionary Act&#8221;,27568,2020-07-31T09:43:51,Violence,contribution,keywords,"[2017, October 2019, Today, today, Halloween, ..."


In [23]:
methods.to_csv('data/methods_and_entities.csv', index=False, sep=",")
keywords.to_csv('data/keywords_and_entities.csv', index=False, sep=",")
folksonomies.to_csv('data/folks_and_entities.csv', index=False, sep=",")

In [26]:
#creating a unique dataset
allTaxonomies = pd.concat([methods, keywords,folksonomies])

In [28]:
allTaxonomies.to_csv('data/alltaxonomies_and_entities.csv', index=False, sep=",")

## Planned exercises:
- Design one glyph by isolating and plotting one unique entry
- Finish metaviews on datasets
- Another notebook with entries not repeating based on the taxonomy?